## When trying to study specific solvent molecules of interest in an MD simulation, you may lose track of them due to GROMACS naming conventions. 
### This especially becomes a problem after adding in other solvent molecules and ions to the simulation. In this script, I walk through each step of the simulation prep, and spit out a dataframe at then end which gives us access to the the details of each water of interest so that they are easier to track during the simulation. 





In [13]:
import pandas as pd
import numpy as np
colspecs_pdb = [(0, 6), (6, 11), (12, 16), (16, 17), (17, 20), (21, 22), (22, 26),
            (26, 27), (30, 38), (38, 46), (46, 54), (54, 60), (60, 66), (76, 78),
            (78, 80)]


names_pdb = ['ATOM', 'serial', 'name', 'altloc', 'resname', 'chainid', 'resseq',
         'icode', 'x', 'y', 'z', 'occupancy', 'tempfactor', 'element', 'charge']


colspecs_gro = [(0, 5), (5, 11), (12, 15), (16, 17), (17, 21), (21,22),
            (22, 28), (28, 38), (38, 46)]

names_gro = ['Resname', 'serial', 'name', 'altloc', 'somenum', 'chainid',
         'x', 'y', 'z']

## First, load in the .pdb file from original expanded structure.  This gives information about EVERY single chain G/g water in P1 unit cell
### FYI: (Resseq=ID)

In [14]:
pdb_path = 'MD_files/OOO.pdb'
pdb = pd.read_fwf(pdb_path, names=names_pdb, colspecs=colspecs_pdb)

#Want xyz coordinates to be compatible with .gro
pdb['x'] = pdb['x'].round(decimals=2)/10
pdb['y'] = pdb['y'].round(decimals=2)/10
pdb['z'] = pdb['z'].round(decimals=2)/10
pdb

,ATOM,serial,name,altloc,resname,chainid,resseq,icode,x,y,z,occupancy,tempfactor,element,charge
0,HETATM,50277,O,NaN,HOH,G,1,NaN,-5.178,5.780,35.705,1.0,23.07,O,NaN
1,HETATM,50278,O,NaN,HOH,G,2,NaN,-9.730,5.488,35.861,1.0,32.97,O,NaN
2,HETATM,50279,O,NaN,HOH,G,3,NaN,-7.555,2.863,34.516,1.0,21.38,O,NaN
3,HETATM,50280,O,NaN,HOH,G,4,NaN,-8.282,5.462,33.442,1.0,27.00,O,NaN
4,HETATM,50281,O,NaN,HOH,G,5,NaN,-2.115,6.575,36.542,1.0,34.42,O,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,HETATM,A2AZH,O,NaN,HOH,g,304,NaN,-2.866,13.342,57.549,1.0,36.08,O,NaN
520,HETATM,A2AZI,O,NaN,HOH,g,305,NaN,-3.042,13.122,57.355,1.0,30.67,O,NaN
521,HETATM,A2AZJ,O,NaN,HOH,g,306,NaN,-2.703,13.153,57.378,1.0,27.08,O,NaN
522,HETATM,A2AZK,O,NaN,HOH,g,307,NaN,-2.725,13.875,59.309,1.0,50.00,O,NaN


## Next, load up .gro of full system, and select lines which correspond to the waters of interest

In [15]:
gro_path_2 = 'MD_files/PSII_processed_full.gro'
gro2 = pd.read_fwf(gro_path_2, names=names_gro, colspecs=colspecs_gro, skiprows=2)
justOOO = gro2[436684 - 1572:436684]
just_O= justOOO[justOOO['name'] =='OW']
just_O

,Resname,serial,name,altloc,somenum,chainid,x,y,z
435112,7897.0,HOH,OW,NaN,1,NaN,-5.178,5.780,35.705
435115,7898.0,HOH,OW,NaN,4,NaN,-9.730,5.488,35.861
435118,7899.0,HOH,OW,NaN,7,NaN,-7.555,2.863,34.516
435121,7900.0,HOH,OW,NaN,10,NaN,-8.283,5.462,33.442
435124,7901.0,HOH,OW,NaN,13,NaN,-2.115,6.575,36.542
...,...,...,...,...,...,...,...,...,...
436669,8416.0,HOH,OW,1.0,558,NaN,-2.866,13.342,57.549
436672,8417.0,HOH,OW,1.0,561,NaN,-3.042,13.122,57.355
436675,8418.0,HOH,OW,1.0,564,NaN,-2.703,13.153,57.378
436678,8419.0,HOH,OW,1.0,567,NaN,-2.725,13.875,59.309


## When newbox is generated, coordinates change, but resid stays the same. 

In [30]:
pdb_path_2 = 'MD_files/PSII_processed_OEC_newbox.gro'
pd.options.mode.chained_assignment = None  # default='warn'
pdb2 = pd.read_fwf(pdb_path_2, names=names_gro, colspecs=colspecs_gro, skiprows=2)
justOOO = pdb2[436684 - 1572:436684]
justOOO_O= justOOO['name'] =='OW'
newboxOOO= justOOO[justOOO_O]

oldresid=  np.array(pdb['resseq'])
oldchain=  np.array(pdb['chainid'])
newboxOOO['OldResid'] = np.copy(oldresid)
newboxOOO['OldChainID'] = oldchain

newboxOOO = newboxOOO.drop('somenum', 1)
newboxOOO = newboxOOO.drop('chainid', 1)
newboxOOO

,Resname,serial,name,altloc,x,y,z,OldResid,OldChainID
435112,7897.0,HOH,OW,5.0,6.546,5.844,5.025,1,G
435115,7898.0,HOH,OW,5.0,1.994,5.552,5.181,2,G
435118,7899.0,HOH,OW,5.0,4.169,2.927,3.836,3,G
435121,7900.0,HOH,OW,5.0,3.441,5.526,2.762,4,G
435124,7901.0,HOH,OW,5.0,9.609,6.639,5.862,5,G
...,...,...,...,...,...,...,...,...,...
436669,8416.0,HOH,OW,6.0,8.858,13.406,26.869,304,g
436672,8417.0,HOH,OW,6.0,8.682,13.186,26.675,305,g
436675,8418.0,HOH,OW,6.0,9.021,13.217,26.698,306,g
436678,8419.0,HOH,OW,6.0,8.999,13.939,28.629,307,g


## After adding ions to your system, the resid changes, so you need to match up waters based off of their position

In [31]:
pdb_path_2 = 'MD_files/PSII_processed_OEC_ions.gro'

pdb2 = pd.read_fwf(pdb_path_2, names=names_gro, colspecs=colspecs_gro, skiprows=2)
pdb_O = pdb2['name'] =='OW'
pdb_justO = pdb2[pdb_O]

pdb_justO_x = pdb_justO[pdb_justO.x.isin(newboxOOO.x)] #filter x
pdb_justO_y = pdb_justO_x[pdb_justO_x.y.isin(newboxOOO.y)] #filter y 
gro_justO_z = pdb_justO_y[pdb_justO_y.z.isin(newboxOOO.z)] #filter z 


## Final step: Figure out if any of your special waters of interest were replaced by an ion

In [32]:
oldresnum=[]
oldchainid=[]
ressiq = [] 
for i, row in gro_justO_z.iterrows():
   # print(i, row)
    for p, raw in newboxOOO.iterrows():
        if (newboxOOO.at[p,'x'] == gro_justO_z.at[i, 'x']) and (newboxOOO.at[p,'y'] == gro_justO_z.at[i, 'y']) and (newboxOOO.at[p,'z'] == gro_justO_z.at[i, 'z']):
            oldresnum.append(newboxOOO.at[p,'Resname'])
            oldchainid.append(newboxOOO.at[p, 'OldChainID'])
            ressiq.append(newboxOOO.at[p, 'OldResid'])

            break           
gro_justO_z['OldResNumBeforeIONS']=oldresnum
gro_justO_z['OriginalXTALChainID']=oldchainid
gro_justO_z['OriginalXTALResid']= ressiq

In [33]:
finalgro = gro_justO_z
print("Number of XTAL waters lost = %d" %-(len(finalgro) - len(newboxOOO['Resname'])))

list_difference = []
index_difference =[]
missing_waters=[]

list1= list(newboxOOO['Resname'].values)
list2= list(finalgro['OldResNumBeforeIONS'].values)
list3=list(finalgro['OriginalXTALResid'])
for item in list1:
    if item not in list2:
        list_difference.append(item)
        index_difference.append(list1.index(item))
for x in index_difference:
    missing_waters.append(list3[x])
print("Missing waters have resid:", list_difference)
print("This corresponds to waters:", missing_waters)


Number of XTAL waters lost = 2
Missing waters have resid: [7949.0, 8088.0]
This corresponds to waters: [65, 151]


In [34]:
copy1 = list(oldresid[0:131])
copy2 = list(oldresid[131:262])
copy3= list(oldresid[262:262 + 131])
copy4= list(oldresid[262+131 :262 + 131*2])


betterID = [] 
for i, row in finalgro.iterrows():
    if  finalgro.at[i, 'OriginalXTALResid'] in copy1:
        betterID.append(copy1[copy1.index(finalgro.loc[i, 'OriginalXTALResid'])])
        
    if  finalgro.at[i, 'OriginalXTALResid'] in copy2:
        betterID.append(copy1[copy2.index(finalgro.loc[i, 'OriginalXTALResid'])])
        
    if  finalgro.at[i, 'OriginalXTALResid'] in copy3:
        betterID.append(copy1[copy3.index(finalgro.loc[i, 'OriginalXTALResid'])])
        
    if  finalgro.at[i, 'OriginalXTALResid'] in copy4:
        betterID.append(copy1[copy4.index(finalgro.loc[i, 'OriginalXTALResid'])])
finalgro['BETTERID'] = betterID

## Save to dataframe, BETTERID is how you best identify waters in each copy

In [35]:
finalgro.to_pickle('MD_files/WaterIDs.pkl')
finalgro=pd.read_pickle('MD_files/WaterIDs.pkl')
finalgro

,Resname,serial,name,altloc,somenum,chainid,x,y,z,OldResNumBeforeIONS,OriginalXTALChainID,OriginalXTALResid,BETTERID
435016,11086.0,SOL,OW,5.0,17,NaN,6.546,5.844,5.025,7897.0,G,1,1
435019,11087.0,SOL,OW,5.0,20,NaN,1.994,5.552,5.181,7898.0,G,2,2
435022,11088.0,SOL,OW,5.0,23,NaN,4.169,2.927,3.836,7899.0,G,3,3
435025,11089.0,SOL,OW,5.0,26,NaN,3.441,5.526,2.762,7900.0,G,4,4
435028,11090.0,SOL,OW,5.0,29,NaN,9.609,6.639,5.862,7901.0,G,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
436567,11603.0,SOL,OW,6.0,568,NaN,8.858,13.406,26.869,8416.0,g,304,73
436570,11604.0,SOL,OW,6.0,571,NaN,8.682,13.186,26.675,8417.0,g,305,74
436573,11605.0,SOL,OW,6.0,574,NaN,9.021,13.217,26.698,8418.0,g,306,75
436576,11606.0,SOL,OW,6.0,577,NaN,8.999,13.939,28.629,8419.0,g,307,76
